In [1]:
from bs4 import BeautifulSoup
import urllib
from urllib.request import urlopen
import re
import pandas as pd
import requests
from requests import get
import re

# Pulling all links off of the search pages (up to 3000) and putting them in a dataframe to scrape.

In [2]:
url_template="http://www.indeed.com/jobs?q=%22Database+Administrator%22&l=Houston%2C+TX&start={}"
max_results=250
Linkdf=[]

for start in range(0, max_results, 7):
    url=url_template.format(start)
    html=requests.get(url)
    soup=BeautifulSoup(html.content,'html.parser', from_encoding="utf-8")
    
    #for each in soup.find_all(a_="href"):
    page_links=soup.find_all('a',{'href':re.compile("/rc/")})
    for items in page_links:
        Linkdf.append(items['href'])   

In [3]:
#Sanity Check
len(Linkdf)
#print(Linkdf)

284

In [4]:
#This code allows the code to display the full website instead of truncating
pd.set_option('display.max_colwidth',-1)

#Moving it to a data frame
data = {'links':Linkdf}
df = pd.DataFrame(data, columns=['links'])

#append indeed.com to the front of each
df['Web'] = 'https://www.indeed.com'
df['URL'] = df.Web.str.cat(df.links)

#pull out just a list of the websites.
websites=list(df['URL'])

#Sanity Check
#print(websites)
len(websites)

284

In [5]:
websites1=set(websites)

In [6]:
len(websites1)

32

# Looping through websites....

In [7]:
Title=[]
Descriptions=[]
Location=[]
FullDescriptions=[]

for url in websites1:
    response=get(url)
    soup=BeautifulSoup(response.text,'html.parser')
    
    description_containers= soup.find(class_='jobsearch-jobDescriptionText')
    title_containers=soup.find('h3')
    try:
        location_containers=soup.find('',{'class':'jobsearch-CompanyInfoWithoutHeaderImage'}).find_all('div')[-1]
    except:
        location_containers='None Found'
    
    job_descriptions=str(description_containers)
    job_title=str(title_containers.text)
    try:
        locations=str(location_containers.text)
    except AttributeError:
        locations = 'None Found'
    try:
        full_descriptions = str(description_containers.text)
    except AttributeError:
        full_descriptions= 'None Found'
    
    Descriptions.append(job_descriptions)
    Title.append(job_title)
    Location.append(locations)
    FullDescriptions.append(full_descriptions)    

# Extracting what we want from the Descriptions Column

In [8]:
#'Location' left in for sanity check. Should be removed once code is confirmed to work
Descriptions_df = pd.DataFrame(columns = ['Title', 'Location','City', 'State', 'Zip', 'Country', 'Qualifications', 'Skills', 'Responsibilities', 'Education', 'Requirement', 'FullDescriptions'])
Country = ['US', 'USA', 'United States', 'United States of Americal']
States = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA',
          'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND',
          'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

for index, element in enumerate(Descriptions):
    soup=BeautifulSoup(element,'lxml')
    for values in list(Descriptions_df):
        temp_tag = soup.find('b', text=re.compile(values))
        try:
            ul_tag = temp_tag.find_next('ul')
            Descriptions_df.at[index,values] = ul_tag.text
        except AttributeError:
            Descriptions_df.at[index,values]="None Found"
        Descriptions_df.at[index,"Title"]=Title[index]
        Descriptions_df.at[index,"Location"]=Location[index]
        Descriptions_df.at[index,"FullDescriptions"]=FullDescriptions[index]
        words = '|'.join(Country)
        temp = re.findall(words, Location[index])
        if len(temp) != 0:
            Descriptions_df.at[index,"Country"] = temp[0]
        words = '|'.join(States)
        temp = re.findall(words, Location[index])
        if len(temp) != 0:
            Descriptions_df.at[index,"State"] = temp[0]
        temp = re.findall(r'\d+', Location[index])
        if len(temp) != 0:
            Descriptions_df.at[index,"Zip"] = temp[0]  
            
        temp = re.findall(r'[\w w]+,', Location[index])
        if len(temp) != 0:
            Descriptions_df.at[index,"City"] = re.sub(',', '', temp[0])


In [9]:
Descriptions_df

Title  \
0   EPIC Resolute Hospital Billing Clinical Application Support Analyst IV       
1   Pulse Fee Billing Database Administrator                                     
2   Fund Accounting Manager                                                      
3   Sales Operations Administrator                                               
4   Senior Specialist - SQL DBA                                                  
5   Principal Database Administrator - IT                                        
6   Systems Administrator II                                                     
7   Data Manager/Project Coordinator III                                         
8   Secretary - Executive Director of Assessment and Evaluation                  
9   Database Administrator                                                       
10  Technical Lab Coordinator                                                    
11  Senior Analyst Programmer II                                                 
12  Database Administrator – Informatica                                         
13  EPIC Resolute Professional Billing Clinical Application Support Analyst IV   
14  NoSQL Database Administrator                                                 
15  Admissions Representative                                                    
16  Operations Coordinator                                                       
17  Senior Database Administrator                                                
18  Operational Support Administrator                                            
19  Database Administrator                                                       
20  Fund Accounting Assistant Manager                                            
21  Database Administrator                                                       
22  ETL/Data Developer (Microsoft BI Stack Experience only)                      
23  SAP Security-A Roles Database Administrator                                  
24  Junior Database Administrator                                                
25  Principal Database Administrator - Information Technology                    
26  IT Network Manager                                                           
27  Sr. Subcontracts Administrator                                               
28  SENIOR ADMINISTRATIVE ASSISTANT                                              
29  HR Operations Manager (m/w/d)                                                
30  SQL Server DBA                                                               
31  Lunar Gateway Space Vehicle Data Software Developer                          

             Location         City State         Zip     Country  \
0   Houston, TX        Houston      TX    None Found  None Found   
1   Houston, TX        Houston      TX    None Found  None Found   
2   Houston, TX 77046  Houston      TX    77046       None Found   
3   Houston, TX        Houston      TX    None Found  None Found   
4   Houston, TX        Houston      TX    None Found  None Found   
5   Houston, TX 77002  Houston      TX    77002       None Found   
6   Houston, TX 77036  Houston      TX    77036       None Found   
7   Houston, TX        Houston      TX    None Found  None Found   
8   League City, TX    League City  TX    None Found  None Found   
9   Houston, TX        Houston      TX    None Found  None Found   
10  Baytown, TX 77520  Baytown      TX    77520       None Found   
11  League City, TX    League City  TX    None Found  None Found   
12  Houston, TX        Houston      TX    None Found  None Found   
13  Houston, TX        Houston      TX    None Found  None Found   
14  Houston, TX        Houston      TX    None Found  None Found   
15  Houston, TX 77024  Houston      TX    77024       None Found   
16  Houston, TX        Houston      TX    None Found  None Found   
17  Houston, TX 77089  Houston      TX    77089       None Found   
18  Houston, TX        Houston      TX    None Found  None Found   
19  We

# Move to csv

In [10]:
Descriptions_df.to_csv('Descriptions_df_Database_Houston.csv')